In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    text = re.sub(r'<img.+?>(<br ?/?>)?', '', text)
    text = re.sub(r'^<br ?/?>', '', text)
    return text.strip()

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews")
    if url_normal == None:
        url_normal = soup.find('p', id="regularnews")
    if url_normal == None:
        url_normal = ""
    else:
        url_normal = url_normal.a.get('href')
        if '/http://' in url_normal:
            url_normal = 'http://' + url_normal.split('/http://')[-1]
        else:
            url_normal = 'https://' + url_normal.split('/https://')[-1]
    date = soup.find('p', id="newsDate").text[1:-1]
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

# for new web easy
def scrape_easy_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    url_easy = 'https://' + soup.find('meta', property="og:url").get('content').split('/https://')[-1]
    url_normal = soup.find('div', class_="link-to-normal").a.get('href')
    date = soup.find('p', class_="article-main__date").text[1:-1]
    title_easy = soup.find('h1', class_="article-main__title")
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', class_="article-main__body article-body")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', class_="article-main__body article-body").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }



def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    with open('nolinknormal.txt') as f:
        nolink = f.read().split()
    return sorted(set(notyet) - set(nolink))
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

In [ ]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

In [ ]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

In [46]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

# NHK web easy

In [151]:
with open('linkeasy.txt') as f:
    urls = f.read().strip().split()
    ids = set([x.split('.html')[0].split('/')[-1] for x in urls])

id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
print(len(ids), len(id_exist))
print('not yet', len(ids - id_exist))
urls[:10]

918 5742
not yet 0


['https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014345501000/k10014345501000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014358721000/k10014358721000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014360391000/k10014360391000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014364701000/k10014364701000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014407931000/k10014407931000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014420291000/k10014420291000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014425011000/k10014425011000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014425961000/k10014425961000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014450891000/k10014450891000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014466

In [132]:
driver = webdriver.Chrome()

In [150]:
id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
for url in urls:
    ID = url.split('.html')[0].split('/')[-1]
    if ID in id_exist:
        continue
    driver.get(url)
    time.sleep(7)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    archiveurl = 'https://web.archive.org' + snap.group(2)
    response = requests.get(archiveurl)
    if response.status_code == 200:
        html = response.text
    elif response.status_code == 504:
        response = requests.get(archiveurl)
        if response.status_code == 504:
            raise AssertionError
        html = response.text
    time.sleep(3)
    """
    driver.get(archiveurl)
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    """
    try:
        dic = scrape_easy_one(html)
    except:
        dic = scrape_easy_one_new(html)
    js_e(dic)
    id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())

In [145]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
dic

{'id': 'k10015884001000',
 'title_easy': '楽天の優勝セール\u3000安く売っているように見せた疑い',
 'title_easy_ruby': '<ruby>楽天<rt>らくてん</rt></ruby>の<ruby>優勝<rt>ゆうしょう</rt></ruby>セール\u3000<ruby>安<rt>やす</rt></ruby>く<ruby>売<rt>う</rt></ruby>っているように<ruby>見<rt>み</rt></ruby>せた<ruby>疑<rt>うたが</rt></ruby>い',
 'article_easy': 'ことしのプロ野球は<org>東北楽天イーグルス</org>が優勝しました。このチームを持っている会社の<org>楽天</org>は、いろいろな店を集めた「楽天市場」というウェブサイトを運営しています。「楽天市場」ではすぐに優勝セールを始めました。その中には７７％安くして売っている品物もありました。\nしかし、安くなる前の値段をいつもの値段より高く書いて、安くした割合を大きく見せている店があるとインターネットを見た人たちが言ってきました。<org>楽天</org>が調べると、今までに約２０の店が約１０００の品物を安くなったように見せて売っていた疑いがあることが分かりました。\n<org>楽天</org>によると、今回のセールは<org>楽天</org>がよく調べてからインターネットに品物を出すように決めています。しかし、問題になった店は<org>楽天</org>が調べていないのに、インターネットに品物を出していました。このため<org>楽天</org>は、今回問題になった品物を買った客にはお金を返すなどするように店に言うことにしています。',
 'article_easy_ruby': 'ことしのプロ<ruby>野球<rt>やきゅう</rt></ruby>は<span class="colorC"><ruby>東北<rt>とうほく</rt></ruby><ruby>楽天<rt>らくてん</rt></ruby>イーグルス</span>が<ruby>優勝<rt>ゆうしょう</rt></ruby>しました。このチームを<ruby>持<rt>も</rt></ruby>っている<ruby>会社<rt>か

In [146]:
js_e(dic)

In [5]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  8876


[('社会', 2644),
 ('国際', 2383),
 ('科学・文化', 1484),
 ('ビジネス', 1369),
 ('スポーツ', 1075),
 ('政治', 775),
 ('暮らし', 732),
 ('地域', 615),
 ('気象・災害', 414)]

In [4]:
# genre <> keywords
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    